# Woolsey Fire data exploration

In [ ]:
import os
from pathlib import Path

import earthpy as et
import earthpy.plot as ep
import geopandas as gpd
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import numpy as np
from rasterio.enums import Resampling
import rioxarray as rxr
import xarray as xr

from ea_drought_burn.config import CRS
from ea_drought_burn.utils import (
    copy_xr_metadata,
    open_raster,
    plot_bands,
    plot_rgb,
    reproject_match
)




BOUND_KWARGS = {
    "facecolor": "none",
    "edgecolor": "black",
    "linewidth": 2
}
PRISM_CMAP = "summer_r"




# Set working directory to the earthpy data directory
os.chdir(os.path.join(et.io.HOME, "earth-analytics", "data", "woolsey-fire"))

In [ ]:
def plot_four(bands, mask, *bounds, width=16, **kwargs):
    """Plots bands in two columns"""
    
    # Scale fonts
    font_size = plt.rcParamsDefault["font.size"]
    plt.rc("font", size=font_size * width // 16)
    
    # Scale outline
    bound_kwargs = BOUND_KWARGS.copy()
    bound_kwargs["linewidth"] *= width // 16
    
    titles = [mask.format(year) for year in range(2013, 2017)]
    
    nrows = int(np.ceil(len(bands) / 2))
    height = (width * 0.7) / 2 * nrows

    fig, axes = plt.subplots(nrows, 2, figsize=(width, height))
    
    axes_flat = []
    for row in axes:
        try:
            axes_flat.extend(row)
        except TypeError:
            axes_flat.append(row)

    for ax, title, band in zip(axes_flat, titles, bands):
        plot_bands(band, ax=ax, title=title, **kwargs)
        for bound in bounds:
            bound.plot(ax=ax, **bound_kwargs)
    
    # Reset font size to default
    plt.rcParams.update(plt.rcParamsDefault)

# Woolsey Fire boundary

In [ ]:
# Load the Woolsey Fire perimeter
woolsey_fire = gpd.read_file(os.path.join("shapefiles",
                                          "nifc_woolsey_perimeter",
                                          "2018-CAVNC-091023.shp")).to_crs(CRS)

# Set crop boundary
crop_bound = woolsey_fire.envelope

# Set fire boundaries to draw on map
related_fires = woolsey_fire

In [ ]:
# SMM stack is used to reproject other rasters, so load it here
smm_stack = open_raster(
    os.path.join("aviris-climate-vegetation", "SMMDroughtstack.dat"),
    crs=CRS,
    crop_bound=crop_bound
)

# Sentinel-2 pre- and post-fire images

Pre- and post-fire Sentinel-2 imagery is much better than the available Landsat images, which are obscured by smoke or clouds through early 2019.

In [ ]:
# Load pre-fire data from Sentinel (ESA)
path = os.path.join("sentinel-2-imagery",
                    "3_aligned",
                    "L1C_T11SLT_A008633_20181031T184032.tif")
prefire = open_raster(path, crs=CRS)

plot_rgb(prefire, rgb=[3, 2, 1], stretch=True)

In [ ]:
# Load post-fire data from Sentinel
path = os.path.join("sentinel-2-imagery",
                    "3_aligned",
                    "L1C_T11SLT_A018185_20181215T184316.tif")
postfire = open_raster(path, crs=CRS)

plot_rgb(postfire, rgb=[3, 2, 1], stretch=True)

In [ ]:
# Calculate NBR
nir = prefire[7]
swir2 = prefire[11]
pre_nbr = (nir - swir2) / (nir + swir2)
#plot_bands(pre_nbr, cmap="RdYlGn", vmin=-1, vmax=1)

nir = postfire[7]
swir2 = postfire[11]
post_nbr = (nir - swir2) / (nir + swir2)
#plot_bands(post_nbr, cmap="RdYlGn", vmin=-1, vmax=1)

dnbr = pre_nbr - post_nbr

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
plot_bands(dnbr,
           ax=ax,
           cmap="RdYlGn_r",
           title="Sentinel NBR - Woolsey Fire\n Oct 31, 2018 - Dec 15, 2018")
related_fires.plot(ax=ax, **BOUND_KWARGS)

In [ ]:
# Define dNBR classification bins
dnbr_class_bins = [-np.inf, -.1, .1, .27, .66, np.inf]
dnbr_classified = xr.apply_ufunc(np.digitize, dnbr, dnbr_class_bins)

dnbr_classified = dnbr_classified.astype(np.float64)
dnbr_classified = dnbr_classified.rio.clip(woolsey_fire.geometry)

dnbr_cat_names = [
    "Increased Greenness",
    "Unburned to Low",
    "Low",
    "Moderate",
    "High",
]

nbr_colors = [
    "#7FFF00",
    "#006400",
    "#7FFFD4",
    "#FFFF00",
    "#FF0000",
]
nbr_cmap = ListedColormap(nbr_colors)

# Plot the data with a custom legend
fig, ax = plt.subplots(figsize=(10, 8))

plot_bands(dnbr_classified,
           cmap=nbr_cmap,
           vmin=1,
           vmax=5,
           title="Sentinel dNBR - Woolsey Fire\n Oct 31, 2018 - Dec 15, 2018",
           cbar=False,
           scale=False,
           ax=ax)
related_fires.plot(ax=ax, **BOUND_KWARGS)

ep.draw_legend(im_ax=ax.get_images()[0],
               classes=range(5),
               titles=dnbr_cat_names)

# MTBS burn severity plots

MTBS provides normalized, field-validated dNBR plots that are better than the ones I calculated above.

In [ ]:
# Plot MTBS dNBR
path = os.path.join("mtbs-burn-severity",
                    "ca3424011870020181108",
                    "ca3424011870020181108_20171215_20181215_dnbr.tif")
dnbr = open_raster(path, crs=CRS, crop_bound=woolsey_fire.geometry)
dnbr = reproject_match(dnbr, smm_stack[0])
dnbr = dnbr.where(dnbr != -9999, np.nan)

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
plot_bands(dnbr,
           ax=ax,
           cmap="RdYlGn_r",
           title="MTBS dNBR - Woolsey Fire\nDec 15, 2017 - Dec 15, 2018")
related_fires.plot(ax=ax, **BOUND_KWARGS)

In [ ]:
# Plot MTBS classified dNBR
dnbr_class_bins = [-np.inf, 100, 400, 800, 1100, np.inf]
dnbr_classified = xr.apply_ufunc(np.digitize, dnbr, dnbr_class_bins)

dnbr_classified = dnbr_classified.astype(np.float64)
dnbr_classified = dnbr_classified.rio.clip(woolsey_fire.geometry)

dnbr_cat_names = [
    "Unburned to Low",
    "Low",
    "Moderate",
    "High",
    "Increased Greenness"
]

nbr_colors = [
    "#006400",
    "#7FFFD4",
    "#FFFF00",
    "#FF0000",
    "#7FFF00"
]
nbr_cmap = ListedColormap(nbr_colors)

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
plot_bands(dnbr_classified,
           cmap=nbr_cmap,
           vmin=1,
           vmax=5,
           title=("MTBS classified dNBR - Woolsey Fire - 30 m\n"
                  "Dec 15, 2017 - Dec 15, 2018"),
           cbar=False,
           scale=False,
           ax=ax)
related_fires.plot(ax=ax, **BOUND_KWARGS)

ep.draw_legend(im_ax=ax.get_images()[0],
               classes=range(5),
               titles=dnbr_cat_names,
               bbox=(0.01, 0.99))

# Woolsey Fire Stack

The original data included a second stack, WoolseyFireDataStack2, that included some Landsat data and BARC burn severity data, as well as the FAL data included in the SMM stack. All of that content is replicated elsewhere, and this file is not included in the final version of the data for this project.

+ **Landsat:** The WoolseyFireDataStack2 file contains two sets of Landsat files,
  one from 2017-10-24 and one from 2018-10-18. Both pre-date the Woolsey fire and
  include only bands 4 (red) and 7 (SWIR2). None of the spectral indices I'm aware
  of use those two bands.
+ **BARC:** BARC stands for Burned Area Reflectance Classification and is a 
  burn-severity index similar to dNBR. We decided to use the MTBS dNBR data
  instead.

# SMM Stack

The nodata value for the weather/climate data in the SMM stack is -3.40282306e+38. The header file was updated with this value but it was not included in the original file.

In [ ]:
# Plot vegetation community
smm_community = smm_stack[0]

colors = [
    "black",
    "tab:green",
    "tab:olive",
    "tab:cyan",
    "tab:red",
    "tab:blue",
    "lightgray"
]

font_size = plt.rcParamsDefault["font.size"]
plt.rc("font", size=font_size * 2)

fig, ax = plt.subplots(figsize=(20, 20))
plot_bands(smm_community,
           ax=ax,
           title="SMM vegetation community map",
           cmap=ListedColormap(colors),
           cbar=False)

bound_kwargs = BOUND_KWARGS.copy()
bound_kwargs["linewidth"] *= 2
related_fires.plot(ax=ax, **bound_kwargs)

# Draw legend
labels = [
    "No data",
    "Annual grass",
    "Chaparral",
    "Coastal sage scrub",
    "Oak woodland",
    "Riparian",
    "Substrate"
]
ep.draw_legend(ax.get_images()[0], 
               titles=labels,
               classes=range(len(labels)))

plt.rcParams.update(plt.rcParamsDefault)

In [ ]:
# Plot fraction alive
fal = smm_stack[1:5].copy()

plot_four(fal,
          "Fraction alive ({})",
          related_fires,
          width=64,
          cmap="PiYG",
          vmin=0,
          vmax=1)

In [ ]:
# Plot year-on-year difference in fraction alive

dfal_13_14 = fal[0] - fal[1]
dfal_14_15 = fal[1] - fal[2]
dfal_15_16 = fal[2] - fal[3]
dfal_13_16 = fal[0] - fal[3]

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(64, 48))

font_size = plt.rcParamsDefault["font.size"]
plt.rc("font", size=font_size * 4)

plot_bands(dfal_13_14, ax=ax1, cmap="PiYG_r", vmin=-1, vmax=1, title="dFAL (2013-2014)")
plot_bands(dfal_14_15, ax=ax2, cmap="PiYG_r", vmin=-1, vmax=1, title="dFAL (2014-2015)")
plot_bands(dfal_15_16, ax=ax3, cmap="PiYG_r", vmin=-1, vmax=1, title="dFAL (2015-2016)")
plot_bands(dfal_13_16, ax=ax4, cmap="PiYG_r", vmin=-1, vmax=1, title="dFAL (2013-2016)")

bound_kwargs = BOUND_KWARGS.copy()
bound_kwargs["linewidth"] *= 4
for ax in (ax1, ax2, ax3, ax4):
    related_fires.plot(ax=ax, **bound_kwargs)

plt.rcParams.update(plt.rcParamsDefault)

In [ ]:
# Plot number of days with precipiation
days_precip = smm_stack[5:9]

plot_four(days_precip, 
          "Days precipitation ({})",
          related_fires,
          cmap=PRISM_CMAP,
          vmin=days_precip.min(),
          vmax=days_precip.max())

In [ ]:
# Plot maximum VPD
max_vpd = smm_stack[9:13]

plot_four(max_vpd, 
          "Max VPD ({})",
          related_fires,
          cmap=PRISM_CMAP,
          vmin=max_vpd.min(),
          vmax=max_vpd.max())

In [ ]:
# Plot minimum temperatue
min_temp = smm_stack[13:17]

plot_four(min_temp, 
          "Minimum temperature ({})",
          related_fires,
          cmap=PRISM_CMAP,
          vmin=min_temp.min(),
          vmax=min_temp.max())

In [ ]:
# Plot number of days over 95 F
heat_days_over_95 = smm_stack[17:21]

plot_four(heat_days_over_95, 
          "Heat days over 95 ({})",
          related_fires,
          cmap="winter",
          vmin=heat_days_over_95.min(),
          vmax=heat_days_over_95.max())

In [ ]:
# Plot cumulative precipitation
cumulative_precip = smm_stack[21:25]
cumulative_precip = cumulative_precip.where(cumulative_precip >= 0, np.nan)

plot_four(cumulative_precip, 
          "Cumulative precipitation ({})",
          related_fires,
          cmap=PRISM_CMAP,
          vmin=cumulative_precip.min(),
          vmax=cumulative_precip.max())

In [ ]:
# Calculate area lost per vegetation type
# FIXME: Check this, numbers differ from previous work

print(f"Total area:   {2089 * 1681 * 15.6 **2 / 1e6:.0f} km2")
print("--")

comm = smm_stack[0]
for i in range(0, 7):
    subset = xr.where(comm == i, True, False).values
    color = colors[i]
    
    # Limit to vegetation (i.e., pixels with FAL calculated)
    comm_13_16 = dfal_13_16.where(subset)
    
    dfal = comm_13_16.mean().item()
    pixels = xr.where(comm_13_16, 1, 0).sum().item()
    pixel_km2 = comm_13_16.rio.resolution()[0] ** 2 * 1e-6
    
    area_km2 = dfal * pixels * pixel_km2
    
    print(labels[i])
    print(f"Pixels:          {pixels}")
    print(f"Area (km2):      {pixels * pixel_km2:.1f} km2")
    print(f"Area (acres):    {pixels * pixel_km2 / 0.004:.1f} acres")
    print(f"Dieback %:       {dfal:.3f}")
    print(f"Dieback (km2):   {area_km2:.1f} km2")
    print(f"Dieback (acres): {area_km2 / 0.004:.1f} acres")
    print("--")
    
    """fig, ax = plt.subplots(figsize=(32, 32))
    plot_bands(dfal_13_16,
               ax=ax,
               title=f"Vegetation community {i} map",
               cmap="PiYG",
               cbar=True)

    bound_kwargs = BOUND_KWARGS.copy()
    bound_kwargs["linewidth"] *= 1
    related_fires.plot(ax=ax, **bound_kwargs)"""


# Dieback threshold

Foster et al. (2017) use a threshold of 0.5431 to distinguish dead pixels from
live pixels based on a comparison of FAL and a field survey of oaks. The
correlation between the aerial and field surveys is poor and only looked at one
of the six vegetation communities in the study area, so that threshold is
poorly constrained. I use a threshold of 0.5 below.

In [ ]:
# Plot dead pixels
half_dead = xr.where(fal < 0.5, True, False)
half_dead = half_dead.rio.clip(woolsey_fire.geometry)
half_dead = half_dead.where(half_dead != -2147483648, False)

plot_four(half_dead,
          ">50% dead ({})",
          related_fires,
          width=64,
          cmap="Greys",
          cbar=False)

# Previous fires

The study area suffered ~24 wildfires between 2000 and 2018. The plot below
shows when each area of the scar burned most recently. Burn severity for previous
fires is not known, which may pose a problem with this calculation given the
large unburned or low severity areas within the Woolsey Fire scar.

In [ ]:
import warnings
import pandas as pd


path = os.path.join("shapefiles",
                    "nifc_related_perimeters",
                    "RELATED_FIRE_PERIMTRS_2000_2018_DD83.shp")
nifc_fires = load_nifc_fires(path, crs=CRS)

nifc_fires.sort_values(["perimeterd"], inplace=True)

nifc_fires

In [ ]:
# Load related fires
path = os.path.join("shapefiles",
                    "nifc_related_perimeters",
                    "RELATED_FIRE_PERIMTRS_2000_2018_DD83.shp")
nifc_fires = load_nifc_fires(path, crs=CRS)
nifc_fires.sort_values(["perimeterd"], inplace=True)

# Build array showing when areas of the scar last burned
last_burned = xr.where(smm_stack[0], -9999, -9999)
last_burned.rio.write_nodata(-9999, inplace=True)

woolsey_date = nifc_fires.iloc[-1].perimeterd
for i, row in nifc_fires.iterrows():
    days = (woolsey_date - row.perimeterd).days
    if days:
    
        # Create a mask based on last_burned with a nodata value of False
        mask = xr.where(last_burned, True, True)
        mask = mask.rio.write_nodata(0) 

        # Update the mask
        mask = mask.rio.clip([row.geometry], drop=False)

        last_burned = copy_xr_metadata(last_burned,
                                       xr.where(mask, days, last_burned))
    
last_burned = last_burned.where(last_burned != -9999, np.nan)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
plot_bands(dnbr_classified,
           cmap=nbr_cmap,
           vmin=1,
           vmax=5,
           title=("MTBS classified dNBR - Woolsey Fire\n"
                  "Dec 15, 2017 - Dec 15, 2018"),
           cbar=False,
           scale=False,
           ax=ax1)
plot_bands(last_burned, ax=ax2, cmap=PRISM_CMAP, title="Days since last fire")
for ax in (ax1, ax2):
    woolsey_fire.plot(ax=ax, **BOUND_KWARGS)